# Notebook

In [13]:
import pandas as pd
import pandas as pd
#from lib.poisson_binning import recommend_k_multiple, bin_and_rate, fit_poisson_trend

In [14]:
df2_valid = pd.read_csv("../1_data/df2_valid.csv")
df3_valid = pd.read_csv("../1_data/df3_valid.csv")

df_beam2 = pd.read_csv("../1_data/beam2.csv")
df_beam3 = pd.read_csv("../1_data/beam3.csv")

In [15]:
# cleaning para solo usar una porción
df2_valid["failsP_acum"] = df2_valid[[f"bitnP{i}" for i in range(0,32)]].sum(axis=1)
df3_valid["failsP_acum"] = df3_valid[[f"bitnP{i}" for i in range(0,16)]].sum(axis=1)

fails_run2 = df2_valid[["time", "lfsrTMR", "failsP_acum"]]
fails_run3 = df3_valid[["time", "lfsrTMR", "failsP_acum"]]

In [ ]:
    print("--- run2 ---")
    print("Beam Data:")
    print("\tshape:", df_beam2.shape)
    print("\tcolumns:",df_beam2.columns)
    print("Fails Data")
    print("\tshape:", fails_run2.shape)
    print("\tcolumns:",fails_run2.columns)

    print("--- run3 ---")
    print("Beam Data:")
    print("\tshape:", df_beam3.shape)
    print("\tcolumns:",df_beam3.columns)
    print("Fails Data")
    print("\tshape:", fails_run3.shape)
    print("\tcolumns:",fails_run3.columns)


--- run2 ---
Beam Data:
	shape: (9999, 14)
	columns: Index(['Unnamed: 0', 'time', 'TID', 'HEH', 'N1MeV', 'run_group', 'dt', 'dTID',
       'dHEH', 'dN1MeV', 'TID_dose_rate', 'N1MeV_dose_rate', 'HEH_dose_rate',
       'beam_on'],
      dtype='object')
Fails Data
	shape: (1030123, 3)
	columns: Index(['time', 'lfsrTMR', 'failsP_acum'], dtype='object')
--- run3 ---
Beam Data:
	shape: (9997, 14)
	columns: Index(['Unnamed: 0', 'time', 'TID', 'HEH', 'N1MeV', 'run_group', 'dt', 'dTID',
       'dHEH', 'dN1MeV', 'TID_dose_rate', 'N1MeV_dose_rate', 'HEH_dose_rate',
       'beam_on'],
      dtype='object')
Fails Data
	shape: (1081006, 3)
	columns: Index(['time', 'lfsrTMR', 'failsP_acum'], dtype='object')


##  Poisson

In [37]:
"""
Radiation reliability binning + Poisson statistics (hardened)

Key features vs v1:
- Robust time parsing (strings / datetime64 / epoch seconds)
- Reset-aware binning, equal-fluence (scaled time) binning, equal-count binning
- Clipped/gated scaled-time to avoid dt_eq explosions (floor + rmax cap)
- Exact Garwood CIs (asymmetric for small-N), configurable alpha
- Optional wall vs beam time for rate denominator
- Area normalization (e.g., run3 16 subsystems → compare to 32)
- Debug/QA utilities: inspectors, conservation checks, GLM trend test (optional)
- Merge sparse bins to guarantee a minimum number of events per bin

Usage sketch:
    res = build_and_summarize(
        df_beam2, fails_run2,
        bin_mode="fluence", n_bins=30,
        flux_col="HEH_dose_rate",
        area_norm=(32, 32),                 # (A_run, A_ref)
        alpha=0.32,                         # 68% CI
        T_source="beam",                   # or "wall"
        min_events_per_bin=5,               # enforce stability
    )

Author: senior-dev pass
"""

from __future__ import annotations
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Optional, Tuple, List, Literal
from scipy.stats import chi2

# -------------------------------
# Time parsing
# -------------------------------
def to_datetime_smart(s: pd.Series) -> pd.Series:
    """Parse time that might be datetime64, ISO string, or epoch seconds (float/int).
    Strategy: try generic parsing; if <90% success, fall back to numeric seconds.
    """
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, errors="coerce")
    s1 = pd.to_datetime(s, errors="coerce")
    if s1.notna().mean() >= 0.9:
        return s1
    s_num = pd.to_numeric(s, errors="coerce")
    return pd.to_datetime(s_num, unit="s", errors="coerce")

# -------------------------------
# Reset detection
# -------------------------------
def detect_resets(
    fails_df: pd.DataFrame,
    time_col: str = "time",
    cum_col: str = "failsP_acum",
    aux_cols: Tuple[str, ...] = ("lfsrTMR",),
    min_gap_s: float = 30.0,
    lfsr_cluster_rate_hz: float = 50.0,
) -> List[pd.Timestamp]:
    """Heuristic, multi-signal reset detection.
    Returns sorted list of boundary timestamps including first/last.
    Signals:
      A) Negative diff in cumulative failures (hard reset)
      B) LFSR edge bursts (clustered edges imply reboot window)
      C) Long inactivity gaps (plateaus)
    """
    f = fails_df.sort_values(time_col).copy()
    f[time_col] = to_datetime_smart(f[time_col]).ffill()

    # A) cumulative drops
    v = pd.to_numeric(f[cum_col], errors="coerce").ffill()
    dv = v.diff()
    candA = f.loc[dv < 0, time_col].tolist()

    # B) lfsr bursts
    candB: List[pd.Timestamp] = []
    for c in aux_cols:
        if c in f.columns:
            l = pd.to_numeric(f[c], errors="coerce")
            edges = (l != l.shift(1)) & l.notna()
            te = f.loc[edges, time_col].astype("int64") / 1e9
            if len(te) > 1:
                dt = np.diff(te)
                streak = (dt < (1.0 / max(lfsr_cluster_rate_hz, 1e-3))).astype(int)
                starts = np.where((streak[1:] == 1) & (streak[:-1] == 0))[0] + 1
                candB.extend(pd.to_datetime(te.iloc[starts], unit="s").tolist())

    # C) long inactivity gaps
    tf = f[time_col].astype("int64") / 1e9
    dts = np.diff(tf)
    candC = f.loc[np.where(dts > min_gap_s)[0], time_col].tolist()

    # Sandwich with bounds
    bounds = [f[time_col].iloc[0]] + sorted(set(candA + candB + candC)) + [f[time_col].iloc[-1]]
    bounds = [t for t in bounds if pd.notna(t)]
    if not bounds:
        return []
    merged = [bounds[0]]
    for t in bounds[1:]:
        if (t - merged[-1]).total_seconds() > min_gap_s:
            merged.append(t)
    if len(merged) == 1 and (f[time_col].iloc[-1] > f[time_col].iloc[0]):
        merged = [f[time_col].iloc[0], f[time_col].iloc[-1]]
    return merged

# -------------------------------
# Scaled time (fluence-equivalent)
# -------------------------------
def compute_scaled_time_clipped(
    beam_df: pd.DataFrame,
    time_col: str = "time",
    dt_col: str = "dt",
    flux_col: Optional[str] = "HEH_dose_rate",
    beam_on_col: Optional[str] = "beam_on",
    ref: Literal["median", "mean", "max"] = "median",
    floor_strategy: Literal["adaptive", "fixed"] = "adaptive",
    min_frac: float = 0.05,
    rmax: float = 1e4,
) -> pd.DataFrame:
    """Compute scaled time to equalize fluence, with floors and caps to prevent explosions.
    - When beam_off, we set scaling ratio=1 (i.e., flux=phi_ref).
    - Floor flux to max(phi_floor, flux) to avoid huge ratios; cap ratio at rmax.
    """
    b = beam_df.sort_values(time_col).copy()
    b[time_col] = to_datetime_smart(b[time_col]).ffill()
    dt = pd.to_numeric(b[dt_col], errors="coerce").fillna(0)

    if (flux_col is not None) and (flux_col in b.columns):
        mask_on = (
            pd.to_numeric(b.get(beam_on_col, 1), errors="coerce").fillna(0) != 0
        )
        flux = pd.to_numeric(b[flux_col], errors="coerce")
        # reference over beam-on samples
        if ref == "median":
            phi_ref = np.nanmedian(flux[mask_on])
        elif ref == "mean":
            phi_ref = np.nanmean(flux[mask_on])
        else:
            phi_ref = np.nanmax(flux[mask_on])
        if not (np.isfinite(phi_ref) and phi_ref > 0):
            phi_ref = 1.0

        if floor_strategy == "fixed":
            phi_floor = min_frac * phi_ref
        else:
            # adaptive: robust floor from lower tail of on-beam distribution
            p10 = np.nanpercentile(flux[mask_on], 10) if mask_on.any() else np.nan
            phi_floor = np.nanmax([
                0.25 * p10 if np.isfinite(p10) else 0.0,
                0.05 * phi_ref,
                1e-12,
            ])

        flux_eff = flux.copy()
        flux_eff[~mask_on] = phi_ref  # no scaling when beam is off
        flux_eff = np.clip(flux_eff, phi_floor, None)

        ratio = (phi_ref / flux_eff).clip(0, rmax)
        b["dt_eq"] = dt * ratio
    else:
        b["dt_eq"] = dt

    b["t_eq"] = b["dt_eq"].cumsum()
    return b

# Legacy simple scaled-time (kept for API compatibility)
def compute_scaled_time(
    beam_df: pd.DataFrame,
    time_col: str = "time",
    dt_col: str = "dt",
    flux_col: Optional[str] = "HEH_dose_rate",
    beam_on_col: Optional[str] = "beam_on",
    ref: Literal["median", "mean", "max"] = "median",
) -> pd.DataFrame:
    return compute_scaled_time_clipped(
        beam_df, time_col, dt_col, flux_col, beam_on_col, ref,
        floor_strategy="adaptive", min_frac=0.05, rmax=1e4,
    )

# -------------------------------
# Events from cumulative
# -------------------------------
def extract_event_times(
    fails_df: pd.DataFrame,
    time_col: str = "time",
    cum_col: str = "failsP_acum",
) -> pd.Series:
    f = fails_df.sort_values(time_col).copy()
    f[time_col] = to_datetime_smart(f[time_col]).ffill()
    c = pd.to_numeric(f[cum_col], errors="coerce").ffill()
    inc = c.diff().fillna(0)
    return f.loc[inc > 0, time_col]

# -------------------------------
# Binning strategies
# -------------------------------
def build_bins_reset_locked(reset_bounds: List[pd.Timestamp], k_multiple: int = 1) -> List[pd.Timestamp]:
    if len(reset_bounds) < 2:
        return reset_bounds
    edges = [reset_bounds[0]]
    for i in range(0, len(reset_bounds) - 1, k_multiple):
        edges.append(reset_bounds[min(i + k_multiple, len(reset_bounds) - 1)])
    out = [edges[0]]
    for e in edges[1:]:
        if e > out[-1]:
            out.append(e)
    return out

def build_bins_equal_fluence(beam_eq: pd.DataFrame, t_eq_col: str = "t_eq", n_bins: int = 20) -> List[pd.Timestamp]:
    b = beam_eq.dropna(subset=[t_eq_col]).copy()
    if b.empty:
        return []
    t0, t1 = b[t_eq_col].iloc[0], b[t_eq_col].iloc[-1]
    edges_eq = np.linspace(t0, t1, n_bins + 1)
    teq = b[t_eq_col].values
    treal = to_datetime_smart(b["time"]).astype("int64") / 1e9
    real_from_eq = np.interp(edges_eq, teq, treal)
    return [pd.to_datetime(x, unit="s") for x in real_from_eq]

def build_bins_equal_count(event_times: pd.Series, target_N: int = 25) -> List[pd.Timestamp]:
    t = pd.to_datetime(event_times).sort_values().reset_index(drop=True)
    if t.empty:
        return []
    idx = list(range(0, len(t), target_N)) + [len(t) - 1]
    edges = [t.iloc[0]] + [t.iloc[i] for i in idx[1:]]
    out = [edges[0]]
    for e in edges[1:]:
        if e > out[-1]:
            out.append(e)
    return out

# -------------------------------
# Poisson rate & CIs per bin
# -------------------------------
@dataclass
class BinStat:
    t_start: pd.Timestamp
    t_end: pd.Timestamp
    N: int
    T: float
    rate: float
    lo: float
    hi: float


def garwood_rate_ci(N: int, T: float, alpha: float = 0.32) -> Tuple[float, float]:
    if T <= 0:
        return (np.nan, np.nan)
    if N == 0:
        return (0.0, (-np.log(alpha)) / T)
    mu_lo = 0.5 * chi2.ppf(alpha / 2, 2 * N)
    mu_hi = 0.5 * chi2.ppf(1 - alpha / 2, 2 * (N + 1))
    return (mu_lo / T, mu_hi / T)


def summarize_bins(
    event_times: pd.Series,
    bin_edges: List[pd.Timestamp],
    timebase_df: Optional[pd.DataFrame] = None,
    use_scaled_time: bool = False,
    T_source: Literal["beam", "wall"] = "beam",
    alpha: float = 0.32,
) -> List[BinStat]:
    """Count events in each bin and compute Poisson rate with exact CI.
    - T_source="beam": use sum(dt or dt_eq) from timebase_df
    - T_source="wall": use wall-clock seconds (ignores timebase df)
    """
    if len(bin_edges) < 2:
        return []
    t = pd.to_datetime(event_times).sort_values().values
    stats: List[BinStat] = []
    for t0, t1 in zip(bin_edges[:-1], bin_edges[1:]):
        N = int(((t >= np.datetime64(t0)) & (t < np.datetime64(t1))).sum())
        if (timebase_df is None) or (T_source == "wall" and not use_scaled_time):
            T = (t1 - t0).total_seconds()
        else:
            b = timebase_df
            b_time = to_datetime_smart(b["time"])  # type: ignore[index]
            m = (b_time >= t0) & (b_time < t1)
            col = "dt_eq" if use_scaled_time else "dt"
            T = (
                pd.to_numeric(b.loc[m, col], errors="coerce").fillna(0).sum()
            )
        rate = N / T if T > 0 else np.nan
        lo, hi = garwood_rate_ci(N, T, alpha=alpha)
        stats.append(BinStat(t0, t1, N, T, rate, lo, hi))
    return stats

# -------------------------------
# Consolidation / orchestration
# -------------------------------
def build_and_summarize(
    df_beam: pd.DataFrame,
    fails_df: pd.DataFrame,
    *,
    bin_mode: Literal["reset", "fluence", "count"] = "reset",
    k_multiple: int = 1,
    n_bins: int = 20,
    target_N: int = 25,
    flux_col: Optional[str] = "HEH_dose_rate",
    area_norm: Optional[Tuple[int, int]] = None,
    alpha: float = 0.32,
    T_source: Literal["beam", "wall"] = "beam",
    scaled_time_fn = compute_scaled_time,  # can swap for compute_scaled_time_clipped
    min_events_per_bin: Optional[int] = None,
) -> pd.DataFrame:
    """High-level helper that returns a tidy per-bin DataFrame.
    - bin_mode: choose binning strategy
    - area_norm: (A_run, A_ref) to scale rates for fair cross-run comparison
    - T_source: "beam" or "wall" denominator
    - scaled_time_fn: inject different scaled-time policies
    - min_events_per_bin: if set, merges adjacent sparse bins until N>=threshold
    """
    beam_eq = scaled_time_fn(df_beam, flux_col=flux_col)
    events = extract_event_times(fails_df)

    if bin_mode == "reset":
        resets = detect_resets(fails_df)
        edges = build_bins_reset_locked(resets, k_multiple=k_multiple)
        use_scaled = False
    elif bin_mode == "fluence":
        edges = build_bins_equal_fluence(beam_eq, n_bins=n_bins)
        use_scaled = True
    else:
        edges = build_bins_equal_count(events, target_N=target_N)
        use_scaled = False

    if len(edges) < 2:
        raise ValueError(
            f"No bins created (len(edges)={len(edges)}). Check time parsing / reset detection. "
            f"First/last: fails=[{fails_df['time'].iloc[0]} .. {fails_df['time'].iloc[-1]}], "
            f"beam=[{df_beam['time'].iloc[0]} .. {df_beam['time'].iloc[-1]}]"
        )

    stats = summarize_bins(
        events, edges, timebase_df=beam_eq, use_scaled_time=use_scaled,
        T_source=T_source, alpha=alpha,
    )
    df_out = pd.DataFrame([s.__dict__ for s in stats])

    # enrich
    df_out["t_mid"] = df_out["t_start"] + (df_out["t_end"] - df_out["t_start"]) / 2
    df_out["width_s"] = (df_out["t_end"] - df_out["t_start"]).dt.total_seconds()

    # area normalization
    if (area_norm is not None) and (len(df_out) > 0):
        A_run, A_ref = area_norm
        scale = (A_ref / A_run) if (A_run and A_ref) else 1.0
        for col in ["rate", "lo", "hi"]:
            df_out[col] = df_out[col] * scale

    # optional: merge sparse bins to guarantee minimum events per bin
    if (min_events_per_bin is not None) and (min_events_per_bin > 0):
        df_out = merge_sparse_bins(df_out, k_min=min_events_per_bin)

    return df_out

# -------------------------------
# QA / Utilities
# -------------------------------
def inspect_scaled_time(beam_eq: pd.DataFrame, label: str, clip_warn_ratio: float = 1e2) -> None:
    dt = pd.to_numeric(beam_eq.get("dt", 0), errors="coerce").fillna(0)
    dt_eq = pd.to_numeric(beam_eq.get("dt_eq", 0), errors="coerce").fillna(0)
    ratio = (dt_eq / dt).replace([np.inf, -np.inf], np.nan)
    ratio = ratio[dt > 0]
    if len(ratio) == 0:
        print(f"[SCALED TIME] {label}: no valid dt rows.")
        return
    q50, q90, q99, mx = np.nanpercentile(ratio, [50, 90, 99, 100])
    print(f"[SCALED TIME] {label}: median={q50:.3g}, p90={q90:.3g}, p99={q99:.3g}, max={mx:.3g}")
    if mx > clip_warn_ratio:
        print(f"⚠️  Large scaling detected (max {mx:.1f}×). Consider tighter floor/cap or gating by beam_on.")


def merge_sparse_bins(df_stats: pd.DataFrame, k_min: int = 5) -> pd.DataFrame:
    rows = df_stats.sort_values("t_start").to_dict("records")
    out: List[dict] = []
    acc = None
    for r in rows:
        if acc is None:
            acc = r
            continue
        if acc["N"] < k_min:
            # merge acc + r
            acc["t_end"] = r["t_end"]
            acc["N"] += r["N"]
            acc["T"] += r["T"]
            acc["rate"] = acc["N"] / acc["T"] if acc["T"] > 0 else np.nan
            acc["lo"], acc["hi"] = garwood_rate_ci(acc["N"], acc["T"], alpha=0.32)
            acc["t_mid"] = acc["t_start"] + (acc["t_end"] - acc["t_start"]) / 2
            acc["width_s"] = (acc["t_end"] - acc["t_start"]).total_seconds()
        else:
            out.append(acc)
            acc = r
    if acc is not None:
        out.append(acc)
    return pd.DataFrame(out)


def conservation_checks(events: pd.Series, edges: List[pd.Timestamp], beam_eq: pd.DataFrame, use_scaled: bool) -> None:
    t = pd.to_datetime(events).sort_values().values
    Ns, Ts = [], []
    for t0, t1 in zip(edges[:-1], edges[1:]):
        Ns.append(int(((t >= np.datetime64(t0)) & (t < np.datetime64(t1))).sum()))
        b_time = to_datetime_smart(beam_eq["time"])  # type: ignore[index]
        m = (b_time >= t0) & (b_time < t1)
        Ts.append(pd.to_numeric(beam_eq.loc[m, "dt_eq" if use_scaled else "dt"], errors="coerce").fillna(0).sum())
    print(f"Events total: {len(t)} | Sum of bin N: {sum(Ns)}")
    print(f"Sum(T) over bins: {sum(Ts):.3f}")


def check_real_output(df_out: pd.DataFrame, name: str, use_scaled: bool = False, tol_ratio: float = 0.20) -> None:
    print(f"\n[REAL CHECK] {name}")
    req = {"t_start","t_end","N","T","rate","lo","hi","t_mid","width_s"}
    missing = req - set(df_out.columns)
    assert not missing, f"Missing columns: {missing}"
    assert (df_out["t_end"] > df_out["t_start"]).all(), "Found non-positive bin widths."
    assert (df_out["width_s"] > 0).all(), "Non-positive width_s."
    bad_ci = df_out[(df_out["T"] > 0) & ~((df_out["lo"] <= df_out["rate"]) & (df_out["rate"] <= df_out["hi"]))]
    assert bad_ci.empty, f"Rate outside CI in {len(bad_ci)} bins."
    if not use_scaled and len(df_out) > 0:
        S_T = df_out["T"].sum()
        S_W = df_out["width_s"].sum()
        rel = abs(S_T - S_W) / max(S_W, 1e-9)
        print(f"sum(T)={S_T:.3f}, sum(width_s)={S_W:.3f}, rel diff={rel:.3%}")
        assert rel <= tol_ratio, f"sum(T) differs from sum(width_s) by {rel:.1%} (> {tol_ratio:.0%})."


def target_N_for_relative_error(RE: float = 0.2) -> int:
    return int(np.ceil((1.0 / max(RE, 1e-9)) ** 2))

# Optional GLM (requires statsmodels). Kept safe with local import.
def poisson_trend_test(df_stats: pd.DataFrame, x_col: str = "t_mid") -> dict:
    try:
        import statsmodels.api as sm  # type: ignore
    except Exception as e:
        raise RuntimeError("statsmodels is required for poisson_trend_test().") from e
    x = (
        (pd.to_datetime(df_stats[x_col]) - pd.to_datetime(df_stats[x_col]).min())
        .dt.total_seconds()
        .values
        / 3600.0
    )
    y = df_stats["N"].values
    T = df_stats["T"].values
    X = sm.add_constant(x)
    model = sm.GLM(y, X, family=sm.families.Poisson(), offset=np.log(np.clip(T, 1e-12, None)))
    res = model.fit()
    return {
        "slope_per_hour": float(res.params[1]),
        "p_value": float(res.pvalues[1]),
        "AIC": float(res.aic),
        "summary": res.summary().as_text(),
    }

__all__ = [
    "to_datetime_smart",
    "detect_resets",
    "compute_scaled_time",
    "compute_scaled_time_clipped",
    "extract_event_times",
    "build_bins_reset_locked",
    "build_bins_equal_fluence",
    "build_bins_equal_count",
    "BinStat",
    "garwood_rate_ci",
    "summarize_bins",
    "build_and_summarize",
    "inspect_scaled_time",
    "merge_sparse_bins",
    "conservation_checks",
    "check_real_output",
    "target_N_for_relative_error",
    "poisson_trend_test",
]


## checks

Run on data

In [38]:
# 1) Import or paste the v2 module code from the canvas

# 2) Build scaled time with clipping, then run fluence binning
beam_eq2c = compute_scaled_time_clipped(df_beam2, flux_col="HEH_dose_rate",
                                        floor_strategy="adaptive", rmax=1e4)
inspect_scaled_time(beam_eq2c, "run2 HEH (clipped)")

res_run2_flu = build_and_summarize(
    df_beam=df_beam2, fails_df=fails_run2,
    bin_mode="fluence", n_bins=30,
    flux_col="HEH_dose_rate",
    area_norm=(32,32),
    alpha=0.32,
    T_source="beam",                     # or "wall"
    scaled_time_fn=compute_scaled_time_clipped,
    min_events_per_bin=5,                # optional stability
)

check_real_output(res_run2_flu, "run2 fluence-binned (clipped)", use_scaled=True)


[SCALED TIME] run2 HEH (clipped): median=1, p90=1.78, p99=6.24, max=8.7

[REAL CHECK] run2 fluence-binned (clipped)


median=1 → most intervals had flux ≈ reference flux (no scaling).

p90=1.78 → top 10% of intervals had ~1.8× lower flux than reference.

p99=6.24, max=8.7 → worst intervals scale only ~6–9×. That’s orders-of-magnitude better than before (no Δt_eq explosions). Your floor+cap is doing its job.

In [39]:
N_for_10pct = target_N_for_relative_error(0.10)  # -> 100
res_run2_eqN = build_and_summarize(
    df_beam2, fails_run2, bin_mode="count", target_N=N_for_10pct,
    flux_col="HEH_dose_rate", area_norm=(32,32)
)


In [41]:
res_run2_eqN.head()

,t_start,t_end,N,T,rate,lo,hi,t_mid,width_s
0,2022-09-15 10:48:48.721228,2022-09-15 11:25:24.986096,100,2203.416392,0.045384,0.040877,0.050365,2022-09-15 11:07:06.853662000,2196.264868
1,2022-09-15 11:25:24.986096,2022-09-15 11:28:14.920450,100,165.256229,0.605121,0.545023,0.671527,2022-09-15 11:26:49.953273000,169.934354
2,2022-09-15 11:28:14.920450,2022-09-15 11:31:32.822763,100,220.341639,0.453841,0.408767,0.503645,2022-09-15 11:29:53.871606500,197.902313
3,2022-09-15 11:31:32.822763,2022-09-15 15:00:12.521610,100,12504.388024,0.007997,0.007203,0.008875,2022-09-15 13:15:52.672186500,12519.698847
4,2022-09-15 15:00:12.521610,2022-09-15 15:02:43.820547,100,165.256229,0.605121,0.545023,0.671527,2022-09-15 15:01:28.171078500,151.298937
